In [9]:
# We will need to adjust this to fit our case but this is the structure for the Pipeline

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import PowerTransformer, MinMaxScaler, LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

import pandas as pd
import numpy as np

# Define a method to select attributes from a DataFrame
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

# Read in the tsv file
df = pd.read_csv('06693-0001-Data.tsv', delim_whitespace=True)

# Make a helper dictionary to keep track of 'response', 'numerical', 'categorical'

var_dict = {
    'response': 'V1144',
    'cat_preds': ['V12', 'V13', 'V4518', 'V5114'],
    'num_preds': []
}

# Make a train-test split:
train_set, test_set = train_test_split(df, random_state=1, stratify=df[var_dict['response']])


#num_attribs = var_dict['num_preds']
cat_attribs = var_dict['cat_preds']

# We will need this later
#num_pipeline = Pipeline([
#    ('selector', DataFrameSelector(num_attribs)),
#    ('power_transf', PowerTransformer(method='yeo-johnson', standardize=False)),
#    ('minmax_scaler', MinMaxScaler()),
#])

cat_pipeline = Pipeline([
    ('selector', DataFrameSelector(cat_attribs)),
    ('encoder', OneHotEncoder(sparse=False)),
])

full_pipeline = FeatureUnion(transformer_list=[
    #("num_pipeline", num_pipeline),
    ("cat_pipeline", cat_pipeline),
])

y_train = train_set[var_dict['response']]

# Prepare the training data with the pipeline above
X_train_prepared = cat_pipeline.fit_transform(train_set)
X_train_prepared


C:\Users\dadra\anaconda\Anaconda3\envs\PythonWebMongo\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [11]:
X_train_prepared.shape

(6073, 57)

In [12]:
from sklearn.tree import DecisionTreeClassifier

tree_clf = DecisionTreeClassifier()
tree_clf.fit(X_train_prepared, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [13]:
X_test_prepared = cat_pipeline.fit_transform(test_set)
X_test_prepared.shape

C:\Users\dadra\anaconda\Anaconda3\envs\PythonWebMongo\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(2025, 54)

In [14]:
tree_pred = tree_clf.predict(X_test_prepared)

ValueError: Number of features of the model must match the input. Model n_features is 57 and input n_features is 54 

In [15]:
# So I need to catch where the categorical counts are failing --- in training set finding 57 categories 
# among the 5 variables - in the test set - only 54.

In [16]:
from sklearn.metrics import classification_report
y_train_pred = tree_clf.predict(X_train_prepared)
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       0.65      0.66      0.66      2633
           1       0.65      0.15      0.25       284
           5       0.68      0.72      0.70      3151
           9       1.00      0.20      0.33         5

   micro avg       0.67      0.67      0.67      6073
   macro avg       0.75      0.43      0.49      6073
weighted avg       0.67      0.67      0.66      6073

